# IMAGE AND VIDEO ANALYTICS
## J COMP REVIEW 2
### TEAM MEMBERS:
### SANTOSHRAM M B  (21MIA1137)
### VISHWA D AMRUTH (21MIA1158)
### DWARAKANATH M   (21MIA1160)

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
from pynput.mouse import Button, Controller
import util  # Import the utility functions
import time  # Import time for gesture delay

mouse = Controller()
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=2  # Now detect both hands
)

# Gesture timer for debouncing
gesture_timer = {'Left': {'count': 0, 'last_time': time.time(), 'triggered': False}}

# Function to detect number of fingers for the left hand
def count_fingers(lst):
    cnt = 0
    thresh = (lst.landmark[0].y * 100 - lst.landmark[9].y * 100) / 2
    if (lst.landmark[5].y * 100 - lst.landmark[8].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[9].y * 100 - lst.landmark[12].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[13].y * 100 - lst.landmark[16].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[5].x * 100 - lst.landmark[4].x * 100) > 6:
        cnt += 1
    return cnt 

# Find the index finger tip for the right hand to control the mouse
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        for idx, hand in enumerate(processed.multi_hand_landmarks):
            if processed.multi_handedness[idx].classification[0].label == "Right":  # Right hand only
                index_finger_tip = hand.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
                return index_finger_tip
    return None

# Move the mouse based on index finger of the right hand
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x, y)

# Determine if left or right click for the right hand
def is_left_click(landmark_list, thumb_index_dist):
    return (
        util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
        util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
        thumb_index_dist > 50
    )

def is_right_click(landmark_list, thumb_index_dist):
    return (
        util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
        util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90 and
        thumb_index_dist > 50
    )

# Detect gestures for both mouse (right hand) and keyboard (left hand)
def detect_gesture(frame, landmark_list, processed, handedness):
    global gesture_timer  # To track gesture timing globally
    if len(landmark_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmark_list[4], landmark_list[5]])

        if handedness == "Right" and index_finger_tip is not None:
            # Right hand controls mouse
            if thumb_index_dist < 50 and util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
                move_mouse(index_finger_tip)
            elif is_left_click(landmark_list, thumb_index_dist):
                mouse.press(Button.left)
                mouse.release(Button.left)
                cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif is_right_click(landmark_list, thumb_index_dist):
                mouse.press(Button.right)
                mouse.release(Button.right)
                cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif handedness == "Left":
            # Left hand controls keyboard
            cnt = count_fingers(processed.multi_hand_landmarks[0])  # Finger counting for the left hand
            
            current_time = time.time()
            # Debounce the gesture: check if it's the same gesture and if enough time has passed
            if cnt == gesture_timer['Left']['count']:
                if current_time - gesture_timer['Left']['last_time'] > 0.5:  # Wait for 0.5 seconds
                    if not gesture_timer['Left']['triggered']:  # Avoid repeated triggers
                        if cnt == 1:
                            pyautogui.press("right")  # Press the right arrow key
                            cv2.putText(frame, "Right Arrow", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                        elif cnt == 2:
                            pyautogui.press("left")  # Press the left arrow key
                            cv2.putText(frame, "Left Arrow", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        elif cnt == 3:
                            pyautogui.press("volumeup")  # Press volume up
                            cv2.putText(frame, "Volume Up", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        elif cnt == 4:
                            pyautogui.press("volumedown")  # Press volume down
                            cv2.putText(frame, "Volume Down", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        gesture_timer['Left']['triggered'] = True  # Mark that the action is triggered
            else:
                # Reset the timer and gesture when a new gesture is detected
                gesture_timer['Left'] = {'count': cnt, 'last_time': current_time, 'triggered': False}

def main():
    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            landmark_list = []
            if processed.multi_hand_landmarks:
                for idx, hand in enumerate(processed.multi_hand_landmarks):
                    handedness = processed.multi_handedness[idx].classification[0].label  # Left or Right hand
                    draw.draw_landmarks(frame, hand, mpHands.HAND_CONNECTIONS)
                    for lm in hand.landmark:
                        landmark_list.append((lm.x, lm.y))
                    detect_gesture(frame, landmark_list, processed, handedness)

            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


C:\Users\santoshram m b\miniconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
